## Aufgaben zum Kapitel "Statistik und Wahrscheinlichkeiten"

In [1]:
# invisible
import numpy as np
np.core.arrayprint._line_width = 65
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [13]:
def find_interval(x, 
                  partition, 
                  endpoints=True):
    """ 
    find_interval -> i
    Falls endpoints gleich True ist, ist "i" der Index für den gilt
    partition[i] < x < partition[i+1] gilt, falls solch ein Index 
    existiert. -1 otherwise
        
    Falls endpoints gleich False ist, ist "i" der kleinste
    Index für den x < partition[i] gilt. Falls kein solcher
    Index existiert, len(partition) auf i gesetzt.
    """
    for i in range(0, len(partition)):
        if x < partition[i]:
            return i-1 if endpoints else i
    return -1 if endpoints else len(partition)

In [14]:
import numpy as np

def weighted_choice(sequence, weights):
    """ 
    weighted_choice wählt ein Zufallselement aus 
    'sequence' aus unter Berücksichtigung der
    List bzw. Tupel von Gewichten
    """
    x = np.random.random()
    cum_weights = [0] + list(np.cumsum(weights))
    index = find_interval(x, cum_weights)
    return sequence[index]


In [20]:
def weighted_sample(population, weights, k):
    """ 
    weighted_sample zieht eine Zufallsstichprobe der
    Länge k aus der Sequenz 'population' entsprechend
    der Liste der Gewichte.
    """
    sample = set()
    population = list(population)
    weights = list(weights)
    while len(sample) < k:
        choice = weighted_choice(population, weights)
        sample.add(choice)
        index = population.index(choice)
        weights.pop(index)
        population.remove(choice)
        weights = [ x / sum(weights) for x in weights]
    return list(sample)


In [21]:
def weighted_sample_alternative(population, weights, k):
    """ 
    weighted_sample zieht eine Zufallsstichprobe der
    Länge k aus der Sequenz 'population' entsprechend
    der Liste der Gewichte.
    """
    sample = set()
    population = list(population)
    weights = list(weights)
    while len(sample) < k:
        choice = weighted_choice(population, weights)
        if choice not in sample:
            sample.add(choice)
    return list(sample)

1. Aufgabe.

Wir starten mit einer kleinen Aufgabe mit Würfeln. Beweisen Sie empirisch -- indem Sie ein Simulationsprogramm schreiben -- dass die Wahrscheinlichkeit für das kombinierte Ereignis "Egal welche Zahl gewürfelt wurde-" (E) und "Eine Zahl größer als 2 wurde gewürfelt." 1/3 ist.

In [15]:
from random import randint

outcomes = [ randint(1, 6) for _ in range(10000)]

even_pips = [ x for x in outcomes if x % 2 == 0]
greater_two = [ x for x in outcomes if x > 2]

combined = [ x for x in outcomes if x % 2 == 0 and x > 2]

print(len(even_pips) / len(outcomes))
print(len(greater_two) / len(outcomes))
print(len(combined) / len(outcomes))


0.4983
0.6658
0.3281


2. Aufgabe:

Die Datei ["universities_uk.txt"](universities_uk.txt) beinhaltet eine Liste der Universitäten im Vereinigten Königreich zur Einschreibung zwischen 2013-2014. (Quelle: [Wikipedia](https://en.wikipedia.org/wiki/List_of_universities_in_the_United_Kingdom_by_enrollment#cite_note-1)):


```
Rank Institution Undergraduates Postgraduates Total students
1 Open University in England 	112,535 	10,955 	123,490
2 University of Manchester 	26,485 	11,440 	37,925
3 University of Nottingham 	24,885 	8,385 	33,270
4 Sheffield Hallam University 	25,985 	7,115 	33,100
5 University of Birmingham 	19,185 	13,150 	32,335
6 Manchester Metropolitan University 	26,635 	5,525 	32,160
7 University of Leeds 	23,265 	7,710 	30,975
8 Cardiff University 	21,495 	8,685 	30,180
9 University of South Wales 	23,890 	5,310 	29,195
...
```

Schreiben Sie eine Funktion, die ein Tupel (universities, enrollments, total_number_of_students) zurückliefert mit:

<ul>
<li> universities: Liste der Namen der Universitäten
<li> enrollments: zugehörige Liste mit Einschreibungen
<li> total_number_of_students: Über alle Universitäten
</ul>
<br>
Jetzt können Sie 100000 fiktive Studenten immatrikulieren mit einer Wahrscheinlichkeit, die den "echten" Einschreibungen entspricht.

Wir schreiben zuerst die Funktion "process_datafile", um die Daten aus der Datei zu verarbeiten:


In [16]:
def process_datafile(filename):
    """ process_datafile -> (universities, 
                             enrollments, 
                             total_number_of_students) 
        universities: list of University names
        enrollments: corresponding list with enrollments
        total_number_of_students: over all universities
    """

    universities = []
    enrollments = []
    with open(filename) as fh:
        total_number_of_students = 0
        fh.readline() # get rid of descriptive first line
        for line in fh:
             line = line.strip()
             *praefix, under, post, total = line.rsplit()
             university = praefix[1:]
             total = int(total.replace(",", ""))
             enrollments.append(total)
             universities.append(" ".join(university))
             total_number_of_students += total
    return (universities, enrollments, total_number_of_students)

Lassen wir die Funktion laufen und prüfen das Ergebnis:

In [17]:
universities, enrollments, total_students = \
                       process_datafile("universities_uk.txt")

for i in range(14):
    print(universities[i], end=": ")
    print(enrollments[i])
print("Number of students enrolled in the UK: ", total_students)

Open University in England: 123490
University of Manchester: 37925
University of Nottingham: 33270
Sheffield Hallam University: 33100
University of Birmingham: 32335
Manchester Metropolitan University: 32160
University of Leeds: 30975
Cardiff University: 30180
University of South Wales: 29195
University College London: 28430
King's College London: 27645
University of Edinburgh: 27625
Northumbria University: 27565
University of Glasgow: 27390
Number of students enrolled in the UK:  2299380


Wir wollen einen virtuellen Studenten in eine zufällige Universität einschreiben. Um eine gewichtete Liste zu erhalten, die für die ```weighted_choice``` geeignet ist, müssen wir die Werte der Liste ```enrollments``` normalisieren:

In [18]:
normalized_enrollments = [ students / total_students \
                          for students in enrollments]

# enrolling a virtual student:
print(weighted_choice(universities, normalized_enrollments))

Birmingham City University


Die Aufgabe war 100,000 fiktive Studenten zu "immatrikulieren".
Dies kann mit einer Schleife einfach durchgeführt werden:

In [23]:
# prog4book"
from collections import Counter
from pprint import pprint # schöne Print-Ausgabe

outcomes = []
n = 100000
for i in range(n):
    outcomes.append(weighted_choice(universities, 
                                    normalized_enrollments))

c = Counter(outcomes)
    
pprint(c.most_common(20), indent=2, width=70)

[ ('Open University in England', 5314),
  ('University of Manchester', 1644),
  ('University of Birmingham', 1444),
  ('Sheffield Hallam University', 1411),
  ('Manchester Metropolitan University', 1408),
  ('University of Nottingham', 1384),
  ('University of Leeds', 1346),
  ('University of South Wales', 1308),
  ('Cardiff University', 1250),
  ('University of Edinburgh', 1227),
  ('University of Plymouth', 1222),
  ("King's College London", 1216),
  ('University of Central Lancashire', 1202),
  ('University College London', 1201),
  ('University of the West of England', 1199),
  ('Northumbria University', 1179),
  ('University of Glasgow', 1175),
  ('University of Sheffield', 1160),
  ('Nottingham Trent University', 1139),
  ('University of Oxford', 1127)]


3. Aufgabe:

In dieser Aufgabe wollen wir eine Zeitreise unternehmen. Wir begeben uns  zurück in das antike Pythonia (Πηθωνια). Es war in der Zeit, in der der König Pysseus als gütiger Diktator regierte.
Jene Zeit, als Pysseus seine Botschafter aussandte, um durch die Welt zu reisen und zu verkünden, dass es für seine Prinzen Anacondos (Ανακονδος), Cobrion (Κομπριον), Boatos (Μποατος) und Addokles (Ανδοκλης) an der Zeit sei zu heiraten. Um die geeigneten Kandidatinnen zu finden, veranstaltete Pysseus einen Programmier-Wettbewerb zwischen den ebenso holden wie tapferen Amazonen, besser bekannt als Pythanier aus Pythonia. 11 Amazonen gelang es, in diesem Programmierwettbewerb zu bestehen, wohl auch weil sie die damals noch junge Programmiersprache Python benutzt hatten:

1. Die ätherische Airla (Αιρλα)
2. Barbara (Βαρβαρα), die Eine aus dem fremden Land
3. Eos (Ηως), sieht in der Dämmerung göttlich aus
4. Die süße Glykeria (Γλυκερια)
5. Die anmutige Hanna (Αννα)
6. Helen (Ελενη), das Licht in der Dunkelheit
7. Der gute Engel Agathangelos (Αγαθαγγελος)
8. Die violett getönte Wolke Iokaste (Ιοκαστη)
9. Medousa (Μεδουσα), die Wächterin
10. Die selbstbestimmende Sofronia (Σωφρονια)
11. Andromeda (Ανδρομεδα), die eine, die wie eine Mann oder ein Krieger denkt

Das Los sollte nun entscheiden, welche vier zu Prinzessinnen würden. Zu Beginn hatten alle die gleiche Chance gezogen zu werden. Aus Gründen, die heute nicht mehr bekannt sind, wusste Pysseus jedoch, dass sich die Wahrscheinlichkeiten mit jedem neuen Tag änderten: Sie sank um 1/13 für die ersten 7 Amazonen und stieg um 1/12 für die letzten 4 Amazonen.
Da die letzten vier der Liste auch seinen persönlichen Wünschen entsprachen, beschloss er die Lotterie ein paar Tage in die Zukunft zu  verschieben.

Wie lange musste der König warten, bis er zu 90% sicher sein konnte, dass seine Prinzen Anacondas, Cobrion, Boatos und Addokles die Amazonen Iokaste, Medouse, Sofronia und Andromeda heiraten würden?

Die Sammlung der Amazonen ist als Liste implementiert, während für die Menge aus Pysseusses Favoritinnen auswählen.
Die Gewichtung liegt zu Beginn bei 1/11 für alle, d.h. 1/len(amazons).

Jeder Schleifen-Durchlauf entspricht einem neuen Tag. Jedes Mal, wenn wir einen neuen Durchlauf starten, ziehen wir ```n``` Samples aus den Pythoniern, um das Verhältnis zu berechnen, wie oft die Sample gleich den Favoritinnen des Königs, geteilt durch die Häufigkeit, wie oft die Sample nicht der Idee einer Schwiegertochter entspricht. Dies entspricht der Wahrscheinlichkeit ```prob```. Wir stoppen das erste Mal, wenn die Wahrscheinlichkeit bei 0.9 oder größer liegt.

Die beiden Funktionen ```weighted_same``` und ```weighted_same_alternative``` lassen sich für die Ziehung verwenden.

In [24]:
import time

amazons = ["Airla", "Barbara", "Eos",
           "Glykeria", "Hanna", "Helen",
           "Agathangelos", "Iokaste", 
           "Medousa", "Sofronia", 
           "Andromeda"]

weights = [ 1/len(amazons) for _ in range(len(amazons)) ]

Pytheusses_favorites = {"Iokaste", "Medousa", 
                        "Sofronia", "Andromeda"}
n = 1000
counter = 0

prob = 1 / 330
days = 0
factor1 = 1 / 13
factor2 = 1 / 12

start = time.perf_counter()
while prob < 0.9:
    for i in range(n):
        the_chosen_ones = weighted_sample_alternative(amazons, 
                                                      weights, 
                                                      4)
        if set(the_chosen_ones) == Pytheusses_favorites:
            counter += 1
    prob = counter / n
    counter = 0
    weights[:7] = [ p - p*factor1 for p in weights[:7] ]
    weights[7:] = [ p + p*factor2 for p in weights[7:] ]
    weights = [ x / sum(weights) for x in weights]
    days += 1
print(time.perf_counter() - start)

print("Number of days, he has to wait: ", days)


10.60157794697443
Number of days, he has to wait:  33


Die Werte für die Anzahl der Tage weichen ab, wenn n nicht groß genug ist.

Der folgende Code ist die Lösung ohne Rundungsfehler. Wir verwenden ```Fraction``` aus dem Modul ```fractions```.

In [25]:
import time
from fractions import Fraction


amazons = ["Airla", "Barbara", "Eos",
           "Glykeria", "Hanna", "Helen",
           "Agathangelos", "Iokaste", 
           "Medousa", "Sofronia", 
           "Andromeda"]

weights = [ Fraction(1, 11) for _ in range(len(amazons)) ]

Pytheusses_favorites = {"Iokaste", "Medousa", 
                        "Sofronia", "Andromeda"}
n = 1000
counter = 0

prob = Fraction(1, 330)
days = 0
factor1 = Fraction(1, 13)
factor2 = Fraction(1, 12)

start = time.perf_counter()
while prob < 0.9:
    #print(prob)
    for i in range(n):
        the_chosen_ones = weighted_sample_alternative(amazons, weights, 4)
        if set(the_chosen_ones) == Pytheusses_favorites:
            counter += 1
    prob = Fraction(counter, n)
    counter = 0
    weights[:7] = [ p - p*factor1 for p in weights[:7] ]
    weights[7:] = [ p + p*factor2 for p in weights[7:] ]
    weights = [ x / sum(weights) for x in weights]
    days += 1
print(time.perf_counter() - start)

print("Number of days, he has to wait: ", days)

89.45118331699632
Number of days, he has to wait:  32


Wir können sehen, dass die Lösung mit ```fractions``` schön aber langsam ist.
Dabei spielt die Präzision in unserem Fall keine Rolle.

Jedoch haben wir die Leistung von Python nicht genutzt.
Das machen wir in der nächsten Implementierung:

In [26]:
import time
import numpy as np

amazons = ["Airla", "Barbara", "Eos",
           "Glykeria", "Hanna", "Helen",
           "Agathangelos", "Iokaste", 
           "Medousa", "Sofronia", 
           "Andromeda"]

weights = np.full(11, 1/len(amazons))


Pytheusses_favorites = {"Iokaste", "Medousa", 
                        "Sofronia", "Andromeda"}


n = 1000
counter = 0

prob = 1 / 330
days = 0
factor1 = 1 / 13
factor2 = 1 / 12

start = time.perf_counter()
while prob < 0.9:
    for i in range(n):
        the_chosen_ones = weighted_sample_alternative(amazons, 
                                                      weights, 
                                                      4)
        if set(the_chosen_ones) == Pytheusses_favorites:
            counter += 1
    prob = counter / n
    counter = 0
    weights[:7] = weights[:7] - weights[:7] * factor1
    weights[7:] = weights[7:] + weights[7:] * factor2
    weights = weights / np.sum(weights)
    #print(weights)
    days += 1
print(time.perf_counter() - start)

print("Number of days, he has to wait: ", days)

9.077820878999773
Number of days, he has to wait:  33
